# Simple MNIST convnet
### 一个简单的MNIST卷积网络

原文链接：https://keras.io/examples/vision/mnist_convnet/

## Note:
这个案例其实更应该被当作第一个示例，因为他足够简单，但在这个样例中我会引入一些其他元素来改变程序结构，特别是在数据处理阶段，读者可以对照着原文进行阅读。

----------------

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

------------

## <span id='pd'>Prepare the data</span>
从这部分开始会和原文存在一定出入，在[第一个样例](ImageClassificationFromScratch.ipynb)中使用了 **image_dataset_from_directory** 来构建“流”式数据集，这里同样使用来实现流式的思想。

In [2]:
input_shape = (28, 28, 1)
batch_size = 256
num_classes = 10
epochs = 5

(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)

TensorFlow 提供了 [**from_tensor_slices**](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#from_tensor_slices)方法，该方法能够根据输入的tensor进行切片，并且返回的是一个 **tf.data.Dataset** 类型的数据。  

用这种方式最大的好处在于可以以“数据集”的角度进行处理，而不必考虑data和label的之间的映射关系，后期在图像数据增强上能够极大简化代码量。

In [3]:
train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
    .shuffle(1024)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((x_val, y_val))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
    .shuffle(1024)
)

从上面的代码中可以看出，使用这种方法可以通过链式方法完成数据预处理，以“打乱数据”这个操作为例，在百度中最常见的方法是如下操作：
```python
    import random

    index = [i for i in range(len(data))]   # 生成顺序的索引
    random.shuffle(index)                   # 将顺序索引打乱
    
    data = data[index]                      # 用乱序索引重新排序 data 和 label
    label = label[index]
```

而使用了**tf.data.Dataset**这个对象后，打乱顺序只需要一句简单的命令 **.shuffle(1024)**。在后面的例子中会出现 **.map()** 方法，这个功能更加强大。

------------

## Build the model
在构建模型时，将标准化操作嵌入模型 Rescaling() 层，如果你使用的是GPU那么这要比原文中的操作更好。  

原文中将标准化操作全部给CPU完成了，如下所示：  
```python  
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
```
这意味着你需要确定以下两件事：  
1. 内存有足够的空间来存储这些预处理过后的数据
2. 你不介意IO阻塞的问题

这样的处理在小型数据集如CIFAR-10/100和MNIST上是完全可以的。但在实际应用中，将所有数据一次性读入内存并不现实，这个过程相当漫长并且存在一定风险。  

假设你的数据是10000张PNG格式的图像，一张50KB大小的图片被读入内存中可能会膨胀到2MB大小，这是由于读入时存储类型和图像压缩算法导致的。在内存和硬盘之间有一个swap，当虚拟地址空间不足时会和硬盘进行交换。此时原本保存在硬盘上不足490MB大小的图片集，一次性全部读入内存就可能会变成19.5GB，这种膨胀率是相当恐怖的，你会发现你的硬盘可用空间以肉眼可见的速度减少，当sawp将硬盘可用空间完全占满后仍然不够时，有一定概率导致系统崩溃。所以为了你系统的安全最好从一开始就养成“流”式的想法，每当阅读到这种一次性读入的代码时可以动手将其改造下，多练习几次就可以信手拈来。  

这个思想在本例中体现不明显，因为 **mnist.load_data()** 已经将所有数据全部读入内存了，但最好在初期就养成习惯，这将对后面的应用打下坚实的基础。

回顾第一个案例 [Image classification from scratch](ImageClassificationFromScratch.ipynb) 中使用的 **image_dataset_from_directory** 函数其实也不是一次性将所有数据都读入，函数返回的对象只是记录了图片的存储路径，所以可以发现执行对应的语句几乎是在一瞬间完成的，而如果将一万张图片读入内存是不可能在几秒内完成。  

所以正确的思路是尽量避免因为预处理而引入额外的内存占用，将这些操作打包进模型中让GPU实现。

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Rescaling(1.0/255.0),        # 将标准化流入到模型中
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0

------------

## Train the model

在Tensorflow中有两个容易混淆的[损失函数](https://keras.io/api/losses/)和[评价指标](https://keras.io/api/metrics/)。  

|loss |metrics|one-hot|
|---|---|---|
|SparseCategoricalCrossentropy()|SparseCategoricalAccuracy()|No|
|CategoricalCrossentropy()|CategoricalAccuracy()|Yes|

原文中在[Prepare the data](#pd)部分多了如下操作：
```python  
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
```
这个操作就是one-hot编码，将0-9的数字数字标签映射到了一个 shape=(10, ) 向量上。  
经历过one-hot编码后，标签“1”和“4”分别被编码成如下形式：  
```python  
    label "1": [0,1,0,0,0,0,0,0,0,0]
    label "4": [0,0,0,0,1,0,0,0,0,0]
```

在此处不对标签进行one-hot编码，为的是让读者理解两类损失函数和评价指标之间的差异以及应用场景。  

这里还需要注意的一点是：无论是使用 **SparseCategoricalCrossentropy** 还是 **CategoricalCrossentropy**，在最后一全连接层中神经元的个数应该是的类别总数，而不能认为因为使用了 **SparseCategoricalCrossentropy** 后这个数值变成1。具体可以参考 [链接](https://tensorflow.google.cn/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [5]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)

In [6]:
histroy = model.fit(train_ds, epochs=epochs, verbose=1)

Epoch 1/5
235/235 [==============================] - 4s 7ms/step - loss: 0.4921 - sparse_categorical_accuracy: 0.8543
Epoch 2/5
235/235 [==============================] - 2s 6ms/step - loss: 0.1340 - sparse_categorical_accuracy: 0.9598
Epoch 3/5
235/235 [==============================] - 2s 6ms/step - loss: 0.0952 - sparse_categorical_accuracy: 0.9704
Epoch 4/5
235/235 [==============================] - 2s 6ms/step - loss: 0.0800 - sparse_categorical_accuracy: 0.9753
Epoch 5/5
235/235 [==============================] - 2s 6ms/step - loss: 0.0704 - sparse_categorical_accuracy: 0.9780


------------

## Evaluate the trained model

In [7]:
score = model.evaluate(val_ds, verbose=1)

40/40 [==============================] - 0s 4ms/step - loss: 0.0423 - sparse_categorical_accuracy: 0.9853
